In [ ]:
#THIS FUNCTION IS FOR RUNNING WITH SLURM JOB ARRAY
#(SPLITS UP JOB_ARRAY BELOW INTO EVEN MORE TASKS)
def StartSlurmJobArray(num_jobs,num_slurm_jobs, ISRUN):
    job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    if job_id==0: job_id=1
    if ISRUN==False:
        start_job=1;end_job=num_jobs+1
        return start_job,end_job
    total_elements=num_jobs #total num of variables

    job_range = total_elements // num_slurm_jobs  # Base size for each chunk
    remaining = total_elements % num_slurm_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_slurm_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_slurm_jobs - 1: 
            end_job = total_elements 
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_slurm_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()
    # if sbatch==True:
        
    start_job, end_job = get_job_range(job_id, num_slurm_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    if start_job==0: start_job=1
    if end_job==total_elements: end_job+=1
    return start_job,end_job

# job_id=1
# [start_slurm_job,end_slurm_job,slurm_index_adjust]=StartSlurmJobArray(num_jobs,num_slurm_jobs,ISRUN)
# parcel=parcel1.isel(xh=slice(start_job,end_job))

In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import matplotlib.lines as mlines
import xarray as xr
import os; import time
import pickle
import h5py

In [ ]:
#MAIN DIRECTORIES
mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
scratchDirectory='/home/air673/koa_scratch/'
codeDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Project_Algorithms/Tracking_Algorithms'

In [ ]:
#LOADING DATA
def GetDataDirectories(simulationNumber):
    if simulationNumber == 1:
        Directory=os.path.join(mainDirectory,'Model/cm1r20.3/run')
        res='1km'; t_res='5min'; Np_str='1e6'; Nz_str='34'
    elif simulationNumber == 2:
        Directory=scratchDirectory
        res='1km'; t_res='1min'; Np_str='50e6'; Nz_str='95'
    elif simulationNumber == 3:
        Directory=scratchDirectory
        res='250m'; t_res='1min'; Np_str='50e6'; Nz_str='95'
        
    dataDirectory = os.path.join(Directory, f"cm1out_{res}_{t_res}_{Nz_str}nz.nc")
    parcelDirectory = os.path.join(Directory,f"cm1out_pdata_{res}_{t_res}_{Np_str}np.nc")
    return dataDirectory, parcelDirectory, res,t_res,Np_str,Nz_str
    
def GetData(dataDirectory, parcelDirectory):
    dataNC = xr.open_dataset(dataDirectory, decode_timedelta=True) 
    parcelNC = xr.open_dataset(parcelDirectory, decode_timedelta=True) 
    return dataNC,parcelNC

def SubsetDataVars(dataNC):
    varList = ["thflux", "qvflux", "tsk", "cape", 
               "cin", "lcl", "lfc", "th",
               "prs", "rho", "qv", "qc",
               "qr", "qi", "qs","qg", 
               "buoyancy", "uinterp", "vinterp", "winterp",]
    
    varList += ["ptb_hadv", "ptb_vadv", "ptb_hidiff", "ptb_vidiff",
                "ptb_hturb", "ptb_vturb", "ptb_mp", "ptb_rdamp", 
                "ptb_rad", "ptb_div", "ptb_diss",]
    
    varList += ["qvb_hadv", "qvb_vadv", "qvb_hidiff", "qvb_vidiff", 
                "qvb_hturb", "qvb_vturb", "qvb_mp",]
    
    varList += ["wb_hadv", "wb_vadv", "wb_hidiff", "wb_vidiff",
                "wb_hturb", "wb_vturb", "wb_pgrad", "wb_rdamp", "wb_buoy",]

    return dataNC[varList]

[dataDirectory,parcelDirectory, res,t_res,Np_str,Nz_str] = GetDataDirectories(simulationNumber=1)
[data1,parcel1] = GetData(dataDirectory, parcelDirectory)

In [ ]:
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
job_array=False;index_adjust=0
ocean_fraction=2/8
start_time = time.time();

In [ ]:
#########################################

In [ ]:
########################################
#FUNCTIONS

In [ ]:
#JOB ARRAY SETUP
def StartJobArray(job_id):
    total_elements=len(parcel1['xh']) #total num of variables
    
    if num_jobs >= total_elements:
        raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
    job_range = total_elements // num_jobs  # Base size for each chunk
    remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_jobs - 1: 
            end_job = total_elements #- 1
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()
    
    # job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    # if job_id==0: job_id=1
    start_job, end_job = get_job_range(job_id)
    print(f'\nstart_job = {start_job}, end_job = {end_job}')

    index_adjust=start_job
    return start_job,end_job,index_adjust
# job_id=1
# [start_job,end_job,index_adjust] = StartJobArray(job_id)

In [ ]:
def GetData(start_job,end_job):
    # parcel=parcel1.isel(xh=slice(start_job,end_job))
    
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/OUTPUT/'
    in_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'
    with h5py.File(in_file, 'r') as f:
        # Load the dataset by its name
        W = f['W'][:,start_job:end_job]
        QCQI = f['QCQI'][:,start_job:end_job]
        Z = f['Z'][:,start_job:end_job]
        Y = f['Y'][:,start_job:end_job]
        X = f['X'][:,start_job:end_job]
        parcel_z=f['z'][:,start_job:end_job]

    return W,QCQI,Z,Y,X,parcel_z

def GetLFCData(start_job, end_job):
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/OUTPUT/'
    in_file=dir2+f'lagrangian_LFC_LCL_binary_array_{res}_{t_res}_{Np_str}.h5'
    with h5py.File(in_file, 'r') as f:
        # Load the dataset by its name
        LFC_FULL = f['LFC_FULL'][:,start_job:end_job]
    return LFC_FULL

In [ ]:
#################################
#MORE FUNCTIONS

In [ ]:
def GetSubsetParams(data1):
    #DOMAIN SUBSETTING
    ############################################################
    #FINDING NEW LEFT T-BOUNDARY
    dt=data1['time'][1].item()/1e9 #seconds per timestep
    dhours=(dt/60**2) #hours per timestep
    start_hour=4 #10:00am
    t_start=int(start_hour/dhours)
    #FINDING NEW RIGHT T-BOUNDARY
    end_hour=11 #5pm
    t_end=int(end_hour/dhours)
    #PRINTING
    # print(f't in {t_start}:{t_end}')
    
    
    #FINDING NEW TOP Z-BOUNDARY
    dzh=data1['zh']
    zh_bottom=0; zh_top=np.where(dzh>=19)[0][0]
    dzf=data1['zf']
    zf_bottom=0; zf_top=np.where(dzf>=20)[0][0]
    #PRINTING
    # print(f'zh_top at {zh_bottom}:{zh_top}')
    
    #FINDING THE NEW LEFT X-BOUNDARY
    ocean_percent=2/8
    left_to_coast=data1['xh'][0]+(data1['xh'][-1]-data1['xh'][0])*ocean_percent
    where_coast_xh=np.where(data1['xh']>=left_to_coast)[0][0]#-25
    where_coast_xf=np.where(data1['xf']>=left_to_coast)[0][0]#-25
    #FINDING THE NEW RIGHT X-BOUNDARY
    right_fraction=80/100
    # Normalize to start from zero
    xf = data1['xf'] - data1['xf'][0]
    xh = data1['xh'] - data1['xh'][0]

    # Total physical length
    xf_max = xf[-1]
    xh_max = xh[-1]

    # Find index where physical location exceeds 80% of domain
    end_xf = np.where(xf > right_fraction * xf_max)[0][0]
    end_xh = np.where(xh > right_fraction * xh_max)[0][0]
    #PRINTING
    # print(f'x in {0}:{where_coast_xh-1} FOR SEA')
    # print(f'x in {where_coast_xh}:{end_xh} FOR LAND')
    ############################################################
    return (
        t_start, t_end,
        zh_bottom, zf_bottom,
        zh_top, zf_top,
        where_coast_xh, where_coast_xf,
        end_xh, end_xf
    )
# [t_start, t_end, zh_bottom, zf_bottom, zh_top, zf_top, where_coast_xh, where_coast_xf, end_xh, end_xf] = GetSubsetParams(data1)
# GetSubsetParams(data1)

In [ ]:
# Reading Back Data Later
##################################################################
#DOMAIN SUBSETTING
def DOMAIN_SUBSET(out_arr,index_adjust):
    print(f'length before: {len(out_arr)}')

    [t_start, t_end, zh_bottom, zf_bottom, zh_top, zf_top, where_coast_xh, where_coast_xf, end_xh, end_xf] = GetSubsetParams(data1)
    #SUBSETTING CODE
    ################
    t,p=out_arr[:,1],out_arr[:,0]
    # if 'job_array' in globals():
    #     p -= index_adjust

    #GETTING X VALUES OF EACH PARCEL 
    zs=Z[t,p-index_adjust]
    xs=X[t,p-index_adjust]

    #GETTING SUBSET CONDITIONS
    cond1=(xs>=where_coast_xh)&(xs<=end_xh)
    cond2=(out_arr[:,1]>=t_start)&(out_arr[:,1]<=t_end)
    cond3=(zs>=zh_bottom)&(zs<=zh_top)
    combined_conds=cond1&cond2&cond3

    #SUBSETTING
    where=np.where(combined_conds)
    out_arr=out_arr[where]

    print(f'==> length after: {len(out_arr)}'+'\n')
    return out_arr

In [ ]:
def GetALLArrays_CL(start_job,end_job,index_adjust):
    #LOADING BACK IN
    def load_file():
        in_file=dir+f'Project_Algorithms/Tracking_Algorithms/OUTPUT/parcel_tracking_{res}_{t_res}_{Np_str}.h5'
        with h5py.File(in_file, 'r') as hf:
            out_arr=hf['out_arr'][:]
            save_arr=hf['save_arr'][:]
            save2_arr=hf['save2_arr'][:]
        return out_arr,save_arr,save2_arr
    [out_arr,save_arr,save2_arr]=load_file()
    print(np.where(np.all(out_arr==0,axis=1))) #TESTING#*#*
    
    print('list of first 10 ignored parcels');
    # print(f'there are a total of {len(out_arr)} CL parcels and {len(save_arr)} nonCL parcels'+'\n')
    
    # if 'job_array' in globals():
    #APPLYING JOB_ARRAY TO PARCEL NUMBER
    ####################################
    def job_filter(arr):
        return arr[(arr[:,0]>=start_job)&(arr[:,0]<end_job)]
    print('Applying Job Array')
    out_arr=job_filter(out_arr)
    save_arr=job_filter(save_arr)

    print(np.where(np.all(out_arr==0,axis=1))) #TESTING#*#*
    
    # print(f'there are a total of {len(out_arr)} CL parcels and {len(save_arr)} nonCL parcels'+'\n')
    
    #CHOOSING UNIQUE INDEXES
    ###############################################################################
    def remove_duplicates(arr):
        lst = []
        unique_values, counts = np.unique(arr[:, 0], return_counts=True)
        duplicates = unique_values[counts > 1]
        for elem in duplicates:
            idx = np.where(arr[:, 0] == elem)[0]
            extras = idx[np.where(arr[idx, 1] != np.min(arr[idx, 1]))]
            lst.extend(extras)
        mask = np.ones(len(arr), dtype=bool)
        mask[lst] = False
        return arr[mask]
    out_arr=remove_duplicates(out_arr)
    save_arr=remove_duplicates(save_arr)
    ###############################################################################
    # print(np.where(np.all(out_arr==0,axis=1))) #TESTING
    
    ############################################################ 
    #SUBSETTING
    subset=True
    if subset==True:
        out_arr_subset=DOMAIN_SUBSET(out_arr,index_adjust)
        save_arr_subset=DOMAIN_SUBSET(save_arr,index_adjust)
    ############################################################
    # print(np.where(np.all(out_arr==0,axis=1))) #TESTING
    
    ALL_out_arr=out_arr_subset.copy(); ALL_save_arr=save_arr_subset.copy()
    return ALL_out_arr,ALL_save_arr
# [ALL_out_arr,ALL_save_arr]=GetALLArrays_CL(start_job,end_job,index_adjust)

In [ ]:
def ddt(f,dt=1):
    ddx = (
            f[1:  ]
            -
            f[0:-1]
        ) / (
        2 * dt
    )
    return ddx

#search for deep convective parcels within lagrangian tracking output     
##############################################################
def SHALLOW_threshold(out_arr,zthresh,index_adjust,type):
    
    deep_out_ind=[]; extendrange=[]
    times=data1['time'].values/(1e9 * 60); times=times.astype(float);
    for ind in range(len(out_arr)): 
        # if np.mod(ind,5000)==0: print(f'{ind}/{len(out_arr)}')
        #CHECK TO SSEE IF NEXT MOST LOCAL TIME MAX GOES ABOVE ZTHRESHS 

        #Get Ascending Range Past LFC For Maximum 120 Minutes Simulation Time
        nummins=120; numsteps=int(nummins/times[1])
        aboverange=np.arange(out_arr[ind,2],out_arr[ind,2]+numsteps,1) #range of times between current time and numsteps later
        aboverange=aboverange[aboverange<len(data1['time'])] #caps out at max time
        above=parcel_z[aboverange,out_arr[ind,0]-index_adjust]/1000 #JOBARRAY ADJUST
    
        #Takes The time derivative 
        ddx=ddt(above)

        #Checks whether the Local Time Max Is Located Above zthresh
        signs = np.sign(ddx)
        signs_diff=np.diff(signs)
        local_maxes=np.where((signs_diff != 0) & (signs_diff < 0))[0]+1 #make sure +1 is here
        if len(local_maxes)==0:
            local_maxes=[0]
        elif np.any(above[local_maxes[0]]<=zthresh): #< for SHALLOW, > for DEEP
            extendrange.append(local_maxes[0]) #save to extend xlim of plot later
            deep_out_ind.append(ind)

    #SUBSET OUT FOR FINAL RESULT
    out_arr=out_arr[deep_out_ind,:]
    # print(f'> {zthresh} km. {len(out_arr)} leftover parcels')
    return out_arr#, extendrange
##############################################################

In [ ]:
#search for deep convective parcels within lagrangian tracking output     
##############################################################
def DEEP_threshold(out_arr,zthresh,index_adjust,type):
    
    deep_out_ind=[]; extendrange=[]
    times=data1['time'].values/(1e9 * 60); times=times.astype(float);
    for ind in range(len(out_arr)): 
        # if np.mod(ind,5000)==0: print(f'{ind}/{len(out_arr)}')
        #CHECK TO SSEE IF NEXT MOST LOCAL TIME MAX GOES ABOVE ZTHRESHS 

        #Get Ascending Range Past LFC For Maximum 120 Minutes Simulation Time
        nummins=120; numsteps=int(nummins/times[1])
        aboverange=np.arange(out_arr[ind,2],out_arr[ind,2]+numsteps,1) #range of times between current time and numsteps later
        aboverange=aboverange[aboverange<len(data1['time'])] #caps out at max time
        above=parcel_z[aboverange,out_arr[ind,0]-index_adjust]/1000 #JOBARRAY ADJUST
        
        #Takes The time derivative 
        ddx=ddt(above)

        #Checks whether the Local Time Max Is Located Above zthresh
        signs = np.sign(ddx)
        signs_diff=np.diff(signs)
        local_maxes=np.where((signs_diff != 0) & (signs_diff < 0))[0]+1 #make sure +1 is here
        if len(local_maxes)==0:
            local_maxes=[0]
        
        if np.any(above[local_maxes[0]]>=zthresh): #< for SHALLOW, > for DEEP
            extendrange.append(local_maxes[0]) #save to extend xlim of plot later
            deep_out_ind.append(ind)

    #SUBSET OUT FOR FINAL RESULT
    out_arr=out_arr[deep_out_ind,:]
    # print(f'> {zthresh} km. {len(out_arr)} leftover parcels')
    return out_arr#, extendrange
    # print(out_arr)
##############################################################

In [ ]:
#SHALLOW
def GetSHALLOWArrays_CL(ALL_out_arr,ALL_save_arr,index_adjust):
    convectivelevel=4 #4km
    SHALLOW_out_arr=SHALLOW_threshold(ALL_out_arr,convectivelevel,index_adjust,type='CL')
    SHALLOW_save_arr=SHALLOW_threshold(ALL_save_arr,convectivelevel,index_adjust,type='nonCL')
    # print('list of first 10 SBZ parcels'); print(out_arr[:15])
    # print(f'there are a total of {len(SHALLOW_out_arr)} CL parcels and {len(SHALLOW_save_arr)} nonCL parcels')
    return SHALLOW_out_arr,SHALLOW_save_arr
# [SHALLOW_out_arr,SHALLOW_save_arr]=GetSHALLOWArrays_CL(ALL_out_arr,ALL_save_arr)

In [ ]:
#DEEP
def GetDEEPArrays_CL(ALL_out_arr,ALL_save_arr,index_adjust):    
    convectivelevel=6 #4km
    DEEP_out_arr=DEEP_threshold(ALL_out_arr,convectivelevel,index_adjust,type='CL')
    DEEP_save_arr=DEEP_threshold(ALL_save_arr,convectivelevel,index_adjust,type='nonCL')
    # print('list of first 10 SBZ parcels'); print(out_arr[:15])
    # print(f'there are a total of {len(DEEP_out_arr)} CL parcels and {len(DEEP_save_arr)} nonCL parcels')
    return DEEP_out_arr,DEEP_save_arr
# [DEEP_out_arr,DEEP_save_arr]=GetDEEPArrays_CL(ALL_out_arr,ALL_save_arr)

In [ ]:
def GetArrays_SBZ(ALL_out_arr,index_adjust):
    
    
    def find_SBZ_xmaxs():
        # Define the directory and file path
        if res=='1km':
            dir2 = '/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
        elif res=='250m':
            dir2='/home/air673/koa_scratch/'
        file_path = dir2 + 'Variable_Calculation/OUTPUT/' + 'Convergence' + f'_{res}_{t_res}' + '.h5'
        
        # Open the HDF5 file in read mode
        with h5py.File(file_path, 'r') as f:
            # Access the 'conv' dataset
            conv_dataset = f['conv']
            
            # Define the vertical level you are interested in
            zlev = 4
            
            # Initialize a list to store the xmaxs for each time step
            xmaxs_list = []
    
            # Loop over each time step (axis=0 corresponds to time)
            for t in range(conv_dataset.shape[0]):  # conv_dataset.shape[0] is the time dimension size
                # Read the relevant slice for this time step and vertical level
                Conv_t_zlev = conv_dataset[t, zlev, :, :]  # Shape should be (y_size, x_size)
                
                # Calculate the mean across the y-axis
                Conv_ymean = np.mean(Conv_t_zlev, axis=0)  # Mean across the y-axis
                
                # Find the index of the maximum value along the x-axis
                xmax = np.argmax(Conv_ymean)
                
                # Append the result for this time step
                xmaxs_list.append(xmax)
        
        # Convert the list of xmaxs to a numpy array (optional)
        xmaxs = np.array(xmaxs_list)
    
        return xmaxs #returns SBZ x location for each timestep
    
    
    def subset_SBZ(out_arr):
        xmaxs=find_SBZ_xmaxs()
    
        SBZ_subset=[]
        # test=[] #TESTING
        
        for ind in np.arange(out_arr.shape[0]):
            
            row=out_arr[ind]
            p=row[0]
            t=row[1]
    
            kms=np.argmax(data1['xh'].values-data1['xh'][0].values >= 1)
            if X[t,p-index_adjust] in np.arange( (xmaxs[t]-2*kms),(xmaxs[t]+2*kms) +1):
                SBZ_subset.append(ind)
                # test.append(p) #TESTING
        
        SBZ_out_arr=out_arr[SBZ_subset]
        print(f'there are a total of {len(SBZ_out_arr)} ALL SBZ CL parcels')
    
        valid_range=np.arange(out_arr.shape[0])
        nonSBZ_out_arr=out_arr[list(set(valid_range) - set(SBZ_subset))]
        print(f'there are a total of {len(nonSBZ_out_arr)} ALL nonSBZ CL parcels')
        return SBZ_out_arr,nonSBZ_out_arr
    
    
    # #LOADING CL MAXS FROM CL TRACKING ALGORITHM
    # folder = '/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/Project_Algorithms/Tracking_Algorithms/'
    # whereSBZ=xr.open_dataset(folder+f'whereCL_{res}_ONLY_SBZS.nc').load()
    # whereSBZ=whereSBZ.isel(time=slice(0,len(data1['time'])))
    # whereSBZ=whereSBZ['maxconv_x']
    # def Get_SBZ_X(t,z,y):
    #     Conv_X_Max=whereSBZ[t,z,y,:].values
    #     return Conv_X_Max
    # def subset_SBZ(out_arr):
    
    #     SBZ_subset=[]
    #     # test=[] #TESTING
        
    #     for ind in np.arange(out_arr.shape[0]):
            
    #         row=out_arr[ind]
    #         p=row[0]
    #         t=row[1]
    
    #         kms=np.argmax(data1['xh'].values-data1['xh'][0].values >= 1)
    #         value=X[t,p]
    #         if np.any((value >= xmaxs - 2*kms) & (value <= xmaxs + 2*kms))==True:
    #             SBZ_subset.append(ind)
    #             # test.append(p) #TESTING
        
    #     SBZ_out_arr=out_arr[SBZ_subset]
    #     print(f'there are a total of {len(SBZ_out_arr)} ALL SBZ CL parcels')
    
    #     valid_range=np.arange(out_arr.shape[0])
    #     nonSBZ_out_arr=out_arr[list(set(valid_range) - set(SBZ_subset))]
    #     print(f'there are a total of {len(nonSBZ_out_arr)} ALL nonSBZ CL parcels')
    #     return SBZ_out_arr,nonSBZ_out_arr
    
    
    # print(f'there are a total of {len(SHALLOW_SBZ_out_arr)} SHALLOW SBZ CL parcels')
    # print(f'there are a total of {len(SHALLOW_nonSBZ_out_arr)} SHALLOW nonSBZ CL parcels')
    # print(f'there are a total of {len(DEEP_SBZ_out_arr)} DEEP SBZ CL parcels')
    # print(f'there are a total of {len(DEEP_nonSBZ_out_arr)} DEEP nonSBZ CL parcels')
    
    #SUBSETTING OUT SHALLOW AND DEEP FROM SBZ AND NONSBZ
    [ALL_SBZ_out_arr,ALL_nonSBZ_out_arr]=subset_SBZ(ALL_out_arr)
    SHALLOW_SBZ_out_arr=SHALLOW_threshold(ALL_SBZ_out_arr,4,index_adjust,'SBZ')
    SHALLOW_nonSBZ_out_arr=SHALLOW_threshold(ALL_nonSBZ_out_arr,4,index_adjust,'nonSBZ')
    DEEP_SBZ_out_arr=DEEP_threshold(ALL_SBZ_out_arr,6,index_adjust,'SBZ')
    DEEP_nonSBZ_out_arr=DEEP_threshold(ALL_nonSBZ_out_arr,6,index_adjust,'nonSBZ')

    return ALL_SBZ_out_arr,ALL_nonSBZ_out_arr,SHALLOW_SBZ_out_arr,SHALLOW_nonSBZ_out_arr,DEEP_SBZ_out_arr,DEEP_nonSBZ_out_arr

In [ ]:
def get_ColdPool(out_arr1,out_arr2):
    arr1 = out_arr1[:,0] #CL
    arr2 = out_arr2[:,0] #nonSBZ
    common_values = np.intersect1d(arr1, arr2)
    indices_arr1 = np.where(np.isin(arr1, common_values))[0]  # Indices in arr1
    ColdPool_out_arr=out_arr1[indices_arr1]
    return ColdPool_out_arr

In [ ]:
def SaveData(data_dict, job_id):
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/subsetting/'
    out_file = dir2 + f"parcel_tracking_SUBSET_{res}_{t_res}_{Np_str}_{job_id}" #*#*#

    # Write the data to HDF5 file
    with h5py.File(out_file, 'w') as h5f:
        for key, value in data_dict.items():
            h5f.create_dataset(key, data=value)

#STORING THE MINIMUM CLOUDBASE INFORMATION
def SaveAllCloudBase_Job(all_cloudbase,job_id):
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/subsetting/'
    out_file = dir2 + f"all_cloudbase_{res}_{t_res}_{Np_str}_{job_id}.pkl"
    with open(out_file, 'wb') as f:
        pickle.dump(np.array(all_cloudbase), f)

#STORING THE LFC PROFILE
def SaveLFC_Profile_Job(LFC_profile,job_id):
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/subsetting/'
    out_file = dir2 + f"LFC_Profile_{res}_{t_res}_{Np_str}_{job_id}.pkl"
    with open(out_file, 'wb') as f:
        pickle.dump(np.array(LFC_profile), f)

In [ ]:
counts=[[],[],[],[],[],[]]
def AddCounts(counts,ALL_out_arr,SHALLOW_out_arr,DEEP_out_arr,ALL_save_arr,SHALLOW_save_arr,DEEP_save_arr, job_id):
    counts[0].append(ALL_out_arr.shape[0])
    counts[1].append(SHALLOW_out_arr.shape[0])
    counts[2].append(DEEP_out_arr.shape[0])
    
    counts[3].append(ALL_save_arr.shape[0])
    counts[4].append(SHALLOW_save_arr.shape[0])
    counts[5].append(DEEP_save_arr.shape[0])
    return counts
# AddCounts(counts,ALL_out_arr,SHALLOW_out_arr,DEEP_out_arr,ALL_save_arr,SHALLOW_save_arr,DEEP_save_arr, job_id)

In [ ]:
###################################
# EVEN MORE FUNCTIONS
def compute_after_arrays(data_dict, W, QCQI, index_adjust):
    def find_after_time(out_arr):
        wthresh = 0.5
        qcqithresh = 1e-6
        after_array = np.zeros(len(out_arr), dtype=int)
        for count, out_row in enumerate(out_arr):
            p = out_row[0]
            t2 = out_row[2]
            after = np.where((W[t2:, p - index_adjust] < wthresh) |
                             (QCQI[t2:, p - index_adjust] < qcqithresh))
            if len(after[0]) != 0:
                after_array[count] = after[0][0]
        return after_array

    after_dict = {}
    for key, arr in data_dict.items():
        if "SHALLOW" in key:
            after_dict[key.replace('_arr', '_after_array')] = np.zeros(len(arr), dtype=int)
        else:
            after_dict[key.replace('_arr', '_after_array')] = find_after_time(arr)

    return after_dict
# after_dict = compute_after_arrays(data_dict, W, QCQI, index_adjust)

def AddColumn(data_dict,after_dict):
    for (key1,key2) in zip(data_dict,after_dict):
        data_dict[key1][:, 3] = after_dict[key2]
    return data_dict
# data_dict=AddColumn(data_dict,after_dict)

In [ ]:
def get_mean_cloud_base(out_arr, Z, Y, X, W, QCQI, index_adjust):
    #FINDING MEAN CLOUD BASE 
    zhs=data1['zh'].values
    w_thresh2=0.5
    qcqi_thresh=1e-6
    type='all'
    
    profile_array =np.zeros((len(zhs), 2)) #column 1: var, column 2: counter, column 3: list of zhs
    profile_array[:,1]=zhs;
    
    # cloudbase_lst=[]
    after=4 #20 minutes
    for row in range(out_arr.shape[0]):
        if np.mod(row,3000)==0: print(f'{row}/{out_arr.shape[0]}')
        p=out_arr[row,0]
        
        # ts=np.arange(out_arr[row,4],out_arr[row,5]+1 + after)
        ts_end = min(out_arr[row, 2] + 1 + after, len(data1['time'])) #this takes care of exceeding buffers
        ts = np.arange(out_arr[row, 1], ts_end)
        
        zs=Z[ts,p-index_adjust] #JOBARRAY INDEX_ADJUST
        ys=Y[ts,p-index_adjust] #JOBARRAY INDEX_ADJUST
        xs=X[ts,p-index_adjust] #JOBARRAY INDEX_ADJUST
    
        ws=W[ts,p-index_adjust] #JOBARRAY INDEX_ADJUST
        qcqis=QCQI[ts,p-index_adjust] #JOBARRAY INDEX_ADJUST
        where=np.where((ws>=w_thresh2) & (qcqis>=qcqi_thresh))
        profile_array[zs[where],0]+=1
    del after
    # all_cloudbase=zhs[np.where(profile_array[:,0]!=0)[0][0]]
    nonzero_indices = np.where(profile_array[:, 0] != 0)[0]
    if len(nonzero_indices) > 0:
        all_cloudbase = zhs[nonzero_indices[0]]
    else:
        all_cloudbase = np.nan
    return all_cloudbase

In [ ]:
def get_mean_LFC(out_arr, Z, Y, X, LFC_FULL, index_adjust):
    #FINDING MEAN CLOUD BASE 
    zhs=data1['zh'].values
    w_thresh2=0.5
    qcqi_thresh=1e-6
    type='all'
    
    lfc_array =np.zeros((1, 2)) #column 1: var, column 2: counter, column 3: list of zhs
    
    # cloudbase_lst=[]
    after=4 #20 minutes
    for row in range(out_arr.shape[0]):
        if np.mod(row,3000)==0: print(f'{row}/{out_arr.shape[0]}')
        p=out_arr[row,0]
        
        # ts=np.arange(out_arr[row,4],out_arr[row,5]+1 + after)
        ts_end = min(out_arr[row, 2] + 1 + after, len(data1['time'])) #this takes care of exceeding buffers
        ts = np.arange(out_arr[row, 1], ts_end)
        
        zs=Z[ts,p-index_adjust] #JOBARRAY INDEX_ADJUST
        ys=Y[ts,p-index_adjust] #JOBARRAY INDEX_ADJUST
        xs=X[ts,p-index_adjust] #JOBARRAY INDEX_ADJUST
    
        lfcs=LFC_FULL[ts,p-index_adjust] #JOBARRAY INDEX_ADJUST #*******
        lfcs=lfcs[lfcs>0]
        lfc_array[0,0]+=np.sum(lfcs);lfc_array[0,1]+=len(lfcs)
    del after
    return lfc_array

In [ ]:
count_dict = {key: [] for key in [
    'CL_ALL_out_arr', 'CL_SHALLOW_out_arr', 'CL_DEEP_out_arr',
    'nonCL_ALL_out_arr', 'nonCL_SHALLOW_out_arr', 'nonCL_DEEP_out_arr',

    'SBZ_ALL_out_arr', 'nonSBZ_ALL_out_arr',
    'SBZ_SHALLOW_out_arr', 'nonSBZ_SHALLOW_out_arr',
    'SBZ_DEEP_out_arr', 'nonSBZ_DEEP_out_arr',

    'ColdPool_ALL_out_arr', 'ColdPool_SHALLOW_out_arr',
    'ColdPool_DEEP_out_arr'
]}

In [ ]:
#####################################
#RUNNING

In [ ]:
################################
#JOB ARRAY SETUP
################################
#*#*
# how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100
if Np_str=='1e6':
    num_jobs=60 #1M parcels
    num_slurm_jobs=10
if Np_str=='50e6':
    num_jobs=200 #50M parcels
    num_slurm_jobs=60
##############################

In [ ]:
################################################
#RUNNING

In [ ]:
[start_slurm_job,end_slurm_job]=StartSlurmJobArray(num_jobs=num_jobs,num_slurm_jobs=num_slurm_jobs,ISRUN=True) #if ISRUN is False, then will not run using slurm_job_array
print(f"Running on Slurm_Jobs for Slurm_Job_Ids: {(start_slurm_job,end_slurm_job-1)}")

all_cloudbase=[]
job_id_list=np.arange(start_slurm_job,end_slurm_job)
for job_id in job_id_list:
    if job_id % 10: print(f"current job_id: {job_id}")
    
    [start_job, end_job, index_adjust] = StartJobArray(job_id)
    [W, QCQI, Z, Y, X, parcel_z] = GetData(start_job, end_job) 
    LFC_FULL = GetLFCData(start_job, end_job) 

    #CL vs nonCL
    [CL_ALL_out_arr, nonCL_ALL_out_arr] = GetALLArrays_CL(start_job,end_job,index_adjust)
    [CL_SHALLOW_out_arr, nonCL_SHALLOW_out_arr] = GetSHALLOWArrays_CL(CL_ALL_out_arr, nonCL_ALL_out_arr,index_adjust)
    [CL_DEEP_out_arr, nonCL_DEEP_out_arr] = GetDEEPArrays_CL(CL_ALL_out_arr, nonCL_ALL_out_arr,index_adjust)

    #SBZ vs nonSBZ
    [SBZ_ALL_out_arr, nonSBZ_ALL_out_arr,
     SBZ_SHALLOW_out_arr, nonSBZ_SHALLOW_out_arr,
     SBZ_DEEP_out_arr, nonSBZ_DEEP_out_arr] = GetArrays_SBZ(CL_ALL_out_arr, index_adjust)
    
    # ColdPool
    ColdPool_ALL_out_arr = get_ColdPool(CL_ALL_out_arr, nonSBZ_ALL_out_arr)
    ColdPool_SHALLOW_out_arr = get_ColdPool(CL_SHALLOW_out_arr, nonSBZ_SHALLOW_out_arr)
    ColdPool_DEEP_out_arr = get_ColdPool(CL_DEEP_out_arr, nonSBZ_DEEP_out_arr)

    # Create a dictionary of arrays to save (including SBZ arrays)
    data_dict = {
        'CL_ALL_out_arr': CL_ALL_out_arr,
        'CL_SHALLOW_out_arr': CL_SHALLOW_out_arr,
        'CL_DEEP_out_arr': CL_DEEP_out_arr,
        'nonCL_ALL_out_arr': nonCL_ALL_out_arr,
        'nonCL_SHALLOW_out_arr': nonCL_SHALLOW_out_arr,
        'nonCL_DEEP_out_arr': nonCL_DEEP_out_arr,
    
        'SBZ_ALL_out_arr': SBZ_ALL_out_arr,
        'nonSBZ_ALL_out_arr': nonSBZ_ALL_out_arr,
        'SBZ_SHALLOW_out_arr': SBZ_SHALLOW_out_arr,
        'nonSBZ_SHALLOW_out_arr': nonSBZ_SHALLOW_out_arr,
        'SBZ_DEEP_out_arr': SBZ_DEEP_out_arr,
        'nonSBZ_DEEP_out_arr': nonSBZ_DEEP_out_arr,
    
        'ColdPool_ALL_out_arr': ColdPool_ALL_out_arr,
        'ColdPool_SHALLOW_out_arr': ColdPool_SHALLOW_out_arr,
        'ColdPool_DEEP_out_arr': ColdPool_DEEP_out_arr
    }

    #ADDING ANOTHER COLUMN TO STORE THE AFTER ARRAYS
    for key, arr in data_dict.items():
        new_column = np.zeros((arr.shape[0], 1), dtype=int)
        data_dict[key] = np.hstack((arr, new_column))

    # Compute after-arrays
    after_dict = compute_after_arrays(data_dict, W, QCQI, index_adjust)
    # Adding to Fourth Column
    data_dict=AddColumn(data_dict,after_dict)

    #GETTING THE COUNT
    for key in count_dict:
        count_dict[key].append(data_dict[key].shape[0])

    #GETTING CLOUDBASE ZLEVEL
    cloudbase=get_mean_cloud_base(CL_ALL_out_arr, Z, Y, X, W, QCQI, index_adjust)
    all_cloudbase.append(cloudbase)

    #GETTING LFC PROFILE
    LFC_profile=get_mean_LFC(CL_ALL_out_arr, Z, Y, X, LFC_FULL, index_adjust)
    
    # Call SaveData with the dictionary
    SaveAllCloudBase_Job(all_cloudbase,job_id)
    SaveLFC_Profile_Job(LFC_profile,job_id)
    SaveData({**data_dict, **after_dict}, job_id) # SaveData(data_dict, job_id) 

combined_counts={key: sum(counts) for key, counts in count_dict.items()}
print(combined_counts)

In [ ]:
######################################
#RECOMBINING
recombine=False #KEEP FALSE WHEN RUNNING
# recombine=True

In [ ]:
def ReadData(var_name,job_id):
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/subsetting/'
    out_file=dir2+f"parcel_tracking_SUBSET_{res}_{t_res}_{Np_str}_{job_id}"
    with h5py.File(out_file, 'r') as f:
        out = f[var_name][:]
    return out
# ReadData(var_name,job_id)
def SaveFinalData(dict,out_file):    
    with h5py.File(out_file,'w') as f:
        for key in dict:
            print(key)
            f.create_dataset(key, data=dict[key])

In [ ]:
if recombine==True:
    #GETTING COUNTS FOR MAKING INITIAL RECOMBINED ARRAYS LATER
    count_dict = {key: [] for key in [
        'CL_ALL_out_arr', 'CL_SHALLOW_out_arr', 'CL_DEEP_out_arr',
        'nonCL_ALL_out_arr', 'nonCL_SHALLOW_out_arr', 'nonCL_DEEP_out_arr',
    
        'SBZ_ALL_out_arr', 'nonSBZ_ALL_out_arr',
        'SBZ_SHALLOW_out_arr', 'nonSBZ_SHALLOW_out_arr',
        'SBZ_DEEP_out_arr', 'nonSBZ_DEEP_out_arr',
    
        'ColdPool_ALL_out_arr', 'ColdPool_SHALLOW_out_arr',
        'ColdPool_DEEP_out_arr'
    ]}
    def MakeCount(count_dict):
        print('Getting Tracked Parcel Count')
        for job_id in np.arange(1,num_jobs+1):
            if job_id % 10==0: print(f"current job_id: {job_id}")
            for key in count_dict:
                data_dict_key=ReadData(key,job_id)
                count_dict[key].append(data_dict_key.shape[0])
        combined_counts={key: sum(counts) for key, counts in count_dict.items()}
        return combined_counts
    combined_counts=MakeCount(count_dict)
    print(combined_counts)

In [ ]:
if recombine==True:
    def MakeEmpty(counts_dict):
        empty_dict = {}
        for key, count in counts_dict.items():
            empty_dict[key] = np.zeros((count, 4), dtype=int)
        return empty_dict
    dict=MakeEmpty(combined_counts)

In [ ]:
if recombine==True:
    for job_id in np.arange(1,num_jobs+1):
        if job_id % 10==0: print(f"current job_id: {job_id}")
            
        for key in dict:
            var=ReadData(key,job_id)
            if var.size!=0:
                a=np.where(np.all(dict[key] == 0, axis=1))[0][0]
                b=a+var.shape[0]
                # print(key,a,b) #TESTING
                dict[key][a:b]=var

In [ ]:
if recombine==True:
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    out_file=dir2+f"parcel_tracking_SUBSET_{res}_{t_res}_{Np_str}.h5"
    SaveFinalData(dict,out_file)

In [ ]:
#RECOMBINING AVERAGE CLOUDBASE
if recombine==True:
    #STORING THE MINIMUM CLOUDBASE INFORMATION
    def SaveAllCloudBase_Final(all_cloudbase):
        dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
        out_file = dir2 + f"all_cloudbase_{res}_{t_res}_{Np_str}.pkl"
        with open(out_file, 'wb') as f:
            pickle.dump(np.array(all_cloudbase), f)
    def GetAllCloudBase(job_id):
        dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/'
        in_file=dir2+f'subsetting/all_cloudbase_{res}_{t_res}_{Np_str}_{job_id}.pkl'
        with open(in_file, 'rb') as f:
            cloudbase_job=pickle.load(f)
        return cloudbase_job

    #############
    def CombineCloudBase():
        all_cloudbase=[]
        for job_id in np.arange(1,num_jobs+1):
            all_cloudbase+=list(GetAllCloudBase(job_id))
        all_cloudbase=np.array(all_cloudbase)
        SaveAllCloudBase_Final(all_cloudbase)
        return all_cloudbase
    #############
    #RUNNING
    all_cloudbase=CombineCloudBase()
    print(np.mean(all_cloudbase))

In [ ]:
#RECOMBINING AVERAGE LFC
if recombine==True:
    #STORING THE MINIMUM CLOUDBASE INFORMATION
    def SaveMeanLFC_Final(all_cloudbase):
        dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
        out_file = dir2 + f"MeanLFC_{res}_{t_res}_{Np_str}.pkl"
        with open(out_file, 'wb') as f:
            pickle.dump(np.array(all_cloudbase), f)
    def GetMeanLFCBase(job_id):
        dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/'
        in_file=dir2+f'subsetting/LFC_Profile_{res}_{t_res}_{Np_str}_{job_id}.pkl'
        with open(in_file, 'rb') as f:
            LFC_profile_job=pickle.load(f)
        return LFC_profile_job

    #############
    def CombineCloudBase():
        MeanLFC=np.zeros((1,2))
        for job_id in np.arange(1,num_jobs+1):
            MeanLFC+=GetMeanLFCBase(job_id)
        MeanLFC_Final=(MeanLFC[:,0]/MeanLFC[:,1])[0]
        SaveMeanLFC_Final(MeanLFC_Final)
        return MeanLFC_Final
    #############
    #RUNNING
    MeanLFC_Final=CombineCloudBase()
    print(np.mean(MeanLFC_Final))

In [ ]:
#############################################
#READING BACK IN

In [ ]:
#READING BACK IN
########################
def LoadFinalData(in_file):
    dict = {}
    with h5py.File(in_file, 'r') as f:
        for key in f.keys():
            dict[key] = f[key][:]
    return dict

def LoadAllCloudBase():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    in_file = dir2 + f"all_cloudbase_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        all_cloudbase = pickle.load(f)
    return(all_cloudbase)
min_all_cloudbase=np.nanmin(LoadAllCloudBase())
print(f"Minimum Cloudbase is: {min_all_cloudbase*1000:.0f} m")

def LoadMeanLFC():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    in_file = dir2 + f"MeanLFC_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        MeanLFC = pickle.load(f)
    return MeanLFC
MeanLFC=LoadMeanLFC()
print(f"Mean LFC is: {MeanLFC:.0f} m\n")

dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
in_file=dir2+f"parcel_tracking_SUBSET_{res}_{t_res}_{Np_str}.h5"
final_dict=LoadFinalData(in_file)


#DYNAMICALLY CREATING VARIABLES
for key, value in final_dict.items():
    globals()[key] = value

# #DYNAMICALLY PRINTING VARIABLE SIZES
# for key in final_dict:
#     print(f"{key} has {final_dict[key].shape[0]} parcels")

# PRINTING VARIABLE SIZES (ONE BY ONE)
print(f'ALL: {len(CL_ALL_out_arr)} CL parcels and {len(nonCL_ALL_out_arr)} nonCL parcels')
print(f'SHALLOW: {len(CL_SHALLOW_out_arr)} CL parcels and {len(nonCL_SHALLOW_out_arr)} nonCL parcels')
print(f'DEEP: {len(CL_DEEP_out_arr)} CL parcels and {len(nonCL_DEEP_out_arr)} nonCL parcels')
print('\n')
print(f'ALL: {len(SBZ_ALL_out_arr)} SBZ parcels and {len(nonSBZ_ALL_out_arr)} nonSBZ parcels')
print(f'SHALLOW: {len(SBZ_SHALLOW_out_arr)} SBZ parcels and {len(nonSBZ_SHALLOW_out_arr)} nonSBZ parcels')
print(f'DEEP: {len(SBZ_DEEP_out_arr)} SBZ parcels and {len(nonSBZ_DEEP_out_arr)} nonSBZ parcels')
print('\n')
print(f'ALL: {len(ColdPool_ALL_out_arr)} ColdPool parcels')
print(f'SHALLOW: {len(ColdPool_SHALLOW_out_arr)} ColdPool parcels')
print(f'DEEP: {len(ColdPool_DEEP_out_arr)} ColdPool parcels')


def apply_job_array_filter(start_job, end_job):
    # print("APPLYING JOB ARRAY")

    def job_filter(arr):
        return arr[(arr[:, 0] >= start_job) & (arr[:, 0] < end_job)]

    target_names = [
        'CL_ALL_out_arr', 'nonCL_ALL_out_arr',
        'CL_SHALLOW_out_arr', 'nonCL_SHALLOW_out_arr',
        'CL_DEEP_out_arr', 'nonCL_DEEP_out_arr',
        'SBZ_ALL_out_arr', 'nonSBZ_ALL_out_arr',
        'SBZ_SHALLOW_out_arr', 'nonSBZ_SHALLOW_out_arr',
        'SBZ_DEEP_out_arr', 'nonSBZ_DEEP_out_arr',
        'ColdPool_ALL_out_arr', 'ColdPool_SHALLOW_out_arr', 'ColdPool_DEEP_out_arr'
    ]

    for name in target_names:
        globals()[name+'_2'] = job_filter(globals()[name])

In [ ]:
################################################
#TESTING

In [ ]:
# # dx = 1 km; Np = 1M; Nt = 5 min
# data1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc') #***
# parcel1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc') #***
# res='1km';t_res='5min'
# Np_str='1e6'

# # # dx = 1km; Np = 50M
# # #Importing Model Data
# # dir2='/home/air673/koa_scratch/'
# # data1=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
# # parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc') #***
# # res='1km'; t_res='1min'; Np_str='50e6'

# # # dx = 1km; Np = 50M; Nz = 95
# # #Importing Model Data
# # dir2='/home/air673/koa_scratch/'
# # data1=xr.open_dataset(dir2+'cm1out_1km_1min_95nz.nc') #***
# # parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_95nz.nc') #***
# # res='1km'; t_res='1min_95nz'; Np_str='50e6'

In [ ]:
# #LOADING BACK IN
# def load_file():
#     in_file=dir+f'Project_Algorithms/Tracking_Algorithms/parcel_tracking_{res}_{t_res}_{Np_str}.h5'
#     with h5py.File(in_file, 'r') as hf:
#         out_arr=hf['out_arr'][:]
#         save_arr=hf['save_arr'][:]
#         save2_arr=hf['save2_arr'][:]
#     return out_arr,save_arr,save2_arr
# [out_arr,save_arr,save2_arr]=load_file()
# arr=out_arr.copy()
# # print(np.where(np.all(arr==0,axis=1)))

In [ ]:
# def LoadAllCloudBase():
#     dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/'
#     in_file = dir2 + f"all_cloudbase_{res}_{t_res}_{Np_str}.pkl"
#     with open(in_file, 'rb') as f:
#         all_cloudbase = pickle.load(f)
#     return(all_cloudbase)
# min_all_cloudbase=np.nanmin(LoadAllCloudBase())
# all_cloudbase=min_all_cloudbase
# print(f"Minimum Cloudbase is: {min_all_cloudbase}\n")

In [ ]:
# def LoadMeanLFC():
#     dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/'
#     in_file = dir2 + f"MeanLFC_{res}_{t_res}_{Np_str}.pkl"
#     with open(in_file, 'rb') as f:
#         MeanLFC = pickle.load(f)
#     return MeanLFC
# MeanLFC=LoadMeanLFC()
# print(f"Mean LFC is: {MeanLFC}\n")